```{note}
This code is under active development. Future example will include real data instead of randomly generated data.
```

# Fingerprinting module
-----

## 1. Introduction

Many of currently used statistical analyses rely on the assumption that **brains of similar individuals are relatively homogenous**; meaning that individuals can be grouped together in relative confidence.

However, many studies, mainly in the field of functional magnetic resonance imaging **(REF MUELLER/REFS FP)** have now highlighted that individuals in groups thought to be homogenous show significant within-group differences. This means that

From th

Fingerprinting methodology was originally developped by [Finn et al. (2015)](https://www.nature.com/articles/nn.4135) as a way to identify individuals based on the idea that each person has a unique pattern of functional connectivity preserved across tasks and time. These fingerprints were then associated with various measures (cognition **(REF)**, brain volume **(REF)**, etc.)


In [ ]:
import numpy as np
import pandas as pd

rng = np.random.default_rng()

arr1 = rng.random((5,5))
arr2 = rng.random((5,5))


def random_funct_mat(size):

    rand_arr = rng.random(size)
    upp_triu = np.triu(rand_arr)
    sym_mat = upp_triu + upp_triu.T

    np.fill_diagonal(sym_mat, 1)

    return sym_mat



data_id_age = pd.DataFrame(columns=["id", "age"])

def generate_random_fp_input(size, mod, path_output):
    """ Simple function generating randomized fingerprinting
    inputs of a given size.
    """

    data_id_age = pd.DataFrame(columns=["id", "age"])

    for i in range(0,10):
        id_name = "0" + f"{i+1}" + "a"
        age = np.random.randint(60, 80)

        sym_arr = random_funct_mat(size)

        data_id_age.loc[i, "id"] = id_name
        data_id_age.loc[i, "age"] = age
        np.savetxt(f"{path_output}/fp_{mod}_data/mat_{id_name}.txt", sym_arr, fmt="%1.3f")

    data_id_age.to_csv(f"{path_output}/id_list_{mod}.csv")

generate_random_fp_input((100,100), "mod1", "/Users/stong3/Desktop")